In [1]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient

In [2]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '438'
start = '2013-04-01T00:00:00Z'
end = '2021-05-21T00:00:00Z'
mongo_org_collection = db[str(organization_id)]

In [4]:
def get_dataset():

    labels = ["access_point_addition_request", "antenna_addition_request", "broken_file__file_handling", "frequency_monitor__real_time", "frequency_monitor__survey_inspector", "hardware_failure_customer", "licensing__licence_releases__re_hosting__activations", "notes_customer", "planner__auto_customer", "planner__capacity_customer", "planner__manual_customer", "planner__other", "planner__tools", "reporing__one_click", "reporting__custom_template_customer", "reporting__other_customer", "site_survey_customer", "visualizations_and_analysis", "wow__cad_import", "none_of_the_above"]

    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': {'$in':['web','email', 'api']},
        'original_item.created_at': {'$gte': start, '$lte': end},
        "original_item.custom_fields": { "$ne": 
            { "id": 32258607, "value": None }
        }
    })
    print(tickets.count())
    file_path = os.path.expanduser('~/Downloads/ek_products.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            i = ticket['original_item']
            j = {
                "ticket_id": i['id'],
                "text_a": i["subject"],
                "text_b": i["description"],
                "created_at": i["created_at"],
                "label": [x for x in i['tags'] if x in labels]
            }
            json.dump(j, f)
            _ = f.write('\n')
            count += 1
            if count > 99:
                break
    print("total: ", count)


get_dataset()

<ipython-input-4-3cc08f772254>:14: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(tickets.count())
24474
total:  100
